## ਫਾਈ-4 RAG ਨਾਲ Azure AI Search

ਇਹ ਨੋਟਬੁੱਕ ਦਿਖਾਉਂਦੀ ਹੈ ਕਿ ਫਾਈ-4-ਮਿਨੀ ਅਤੇ ਫਾਈ-4-ਮਲਟੀਮੋਡਲ ਨੂੰ ਰਿਟਰੀਵਲ ਅਗਮੈਂਟਡ ਜਨਰੇਸ਼ਨ (RAG) ਲਈ Azure AI Search ਨਾਲ ਕਿਵੇਂ ਵਰਤਣਾ ਹੈ। ਇਹ ਟੈਕਸਟ-ਮਾਤਰ (ਸਿੰਗਲ-ਮੋਡੈਲਿਟੀ) ਅਤੇ ਟੈਕਸਟ ਅਤੇ ਚਿੱਤਰ (ਮਲਟੀ-ਮੋਡੈਲਿਟੀ) ਦੋਵੇਂ ਸਥਿਤੀਆਂ ਨੂੰ ਕਵਰ ਕਰਦੀ ਹੈ।

**ਪੂਰਵ ਸ਼ਰਤਾਂ:**
*   Azure AI Search ਵੈਕਟਰ ਇੰਡੈਕਸ (ਇੱਕ ਬਣਾਉਣ ਲਈ [ਇਹ ਹਦਾਇਤਾਂ](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) ਦੀ ਪਾਲਣਾ ਕਰੋ)
*   ਫਾਈ-4-ਮਿਨੀ ਜਾਂ ਫਾਈ-4-ਮਲਟੀਮੋਡਲ ਐਂਡਪੋਇੰਟਸ ਜੋ Azure AI Foundry ਵਿੱਚ ਡਿਪਲੌਇ ਕੀਤੇ ਗਏ ਹਨ


In [ ]:
! pip install azure-ai-inference azure-search-documents

### ਟੈਕਸਟ-ਓਨਲੀ RAG ਨਾਲ Phi-4-mini

ਇਸ ਹਿੱਸੇ ਵਿੱਚ ਦਿਖਾਇਆ ਗਿਆ ਹੈ ਕਿ RAG ਲਈ ਸਿਰਫ ਟੈਕਸਟ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਵਰਤਦੇ ਹੋਏ Phi-4-mini ਨੂੰ ਚੈਟ ਕਮਪਲੀਸ਼ਨ ਮਾਡਲ ਵਜੋਂ ਕਿਵੇਂ ਵਰਤਨਾ ਹੈ। ਇਹ ਵਿੱਚ Azure AI Foundry Inference ਅਤੇ Azure AI Search ਨਾਲ ਕਨੈਕਟ ਕਰਨਾ, ਸਬੰਧਤ ਦਸਤਾਵੇਜ਼ਾਂ ਨੂੰ ਪ੍ਰਾਪਤ ਕਰਨਾ, ਅਤੇ ਪ੍ਰਾਪਤ ਸੰਦਰਭ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਜਵਾਬ ਤਿਆਰ ਕਰਨਾ ਸ਼ਾਮਲ ਹੈ।


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### ਫ਼ਿਲੀ-4-ਮਲਟੀਮੋਡਲ ਨਾਲ ਮਲਟੀ-ਮੋਡਲ RAG

ਇਸ ਭਾਗ ਵਿੱਚ ਦਿਖਾਇਆ ਗਿਆ ਹੈ ਕਿ RAG ਲਈ ਫ਼ਿਲੀ-4-ਮਲਟੀਮੋਡਲ ਨੂੰ ਚੈਟ ਪੂਰਨ ਮਾਡਲ ਵਜੋਂ ਕਿਵੇਂ ਵਰਤਣਾ ਹੈ, ਜੋ ਕਿ ਟੈਕਸਟ ਅਤੇ ਚਿੱਤਰ ਦੋਨੋ ਇਨਪੁਟ ਨੂੰ ਸ਼ਾਮਲ ਕਰਦਾ ਹੈ। ਇਹ Azure AI Inference ਅਤੇ Azure AI Search ਨਾਲ ਕਨੈਕਟ ਕਰਨ, ਸਬੰਧਤ ਦਸਤਾਵੇਜ਼ ਪ੍ਰਾਪਤ ਕਰਨ, ਅਤੇ ਮਲਟੀਮੋਡਲ ਜਵਾਬ ਜਨਰੇਟ ਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਨੂੰ ਕਵਰ ਕਰਦਾ ਹੈ।

**ਨੋਟ:** ਜੇ ਤੁਹਾਡੇ Azure AI Search ਇੰਡੈਕਸ ਵਿੱਚ `text_vector` ਅਤੇ `image_vector` ਫੀਲਡ ਹਨ, ਤਾਂ ਤੁਸੀਂ ਮਲਟੀ-ਵੇਕਟਰ ਕਵੈਰੀ ਵੀ ਕਰ ਸਕਦੇ ਹੋ।


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")


---

**ਅਸਵੀਕਾਰਨਾ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੱਜੇਪਣ ਹੋ ਸਕਦੇ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼, ਜੋ ਇਸਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਹੈ, ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
